In [2]:
!pip install joblib
!pip install scikit-learn

import joblib

# Load model and label encoder
clf = joblib.load("knn_sign_model.joblib")
le = joblib.load("label_encoder.joblib")

In [3]:
!pip install mediapipe
!pip install pillow

In [4]:
from PIL import ImageFont, ImageDraw, Image

def draw_chinese_text(frame, text, position, font_path='NotoSansSC-Regular.otf', font_size=48):
    # Convert OpenCV (BGR) to PIL (RGB)
    img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)

    # Load font and draw text
    font = ImageFont.truetype(font_path, font_size)
    draw.text(position, text, font=font, fill=(0, 255, 0))  # Green text

    # Convert back to OpenCV (BGR)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

In [5]:
import mediapipe as mp
import cv2

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)

# Start webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(img_rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            coords = []
            for lm in hand_landmarks.landmark:
                coords.extend([lm.x, lm.y, lm.z])

            if len(coords) == 63:  # 21 points × 3D
                pred = clf.predict([coords])
                label = le.inverse_transform(pred)[0]

                # Show prediction
                cv2.putText(frame, f'{label}', (30, 90),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4)

    # Show frame
    cv2.imshow("Sign Language Prediction", frame)

    # Quit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()